In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf

import math

In [2]:
data=pd.read_csv('~\Downloads\ichimoku.csv')
data

,Unnamed: 0,datetime,high,low,close,open,volume,signal
0,0,2018-01-01,13818.55,12750.00,13135.00,13715.65,6970.441076,0
1,1,2018-01-02,14750.00,12890.02,14580.00,13135.00,16359.347340,0
2,2,2018-01-03,15473.49,14150.00,14749.97,14576.87,19457.238601,0
3,3,2018-01-04,15280.00,13918.04,14811.00,14710.00,18783.876561,0
4,4,2018-01-05,17118.13,14600.00,16579.00,14805.01,22383.657333,0
...,...,...,...,...,...,...,...,...
1468,1468,2022-01-08,42246.37,40501.00,40679.82,41707.04,27031.900870,-1
1469,1469,2022-01-09,42300.00,40520.00,41966.99,40679.82,27291.009660,-1
1470,1470,2022-01-10,42786.70,39650.00,41604.58,41966.99,51064.904360,-1
1471,1471,2022-01-11,43100.00,41158.99,42755.00,41604.58,38062.818280,-1


In [3]:
a=pd.DataFrame(columns=['datetime','signal'])
a['datetime']=data['datetime']
a['signal']=data['signal']
a

,datetime,signal
0,2018-01-01,0
1,2018-01-02,0
2,2018-01-03,0
3,2018-01-04,0
4,2018-01-05,0
...,...,...
1468,2022-01-08,-1
1469,2022-01-09,-1
1470,2022-01-10,-1
1471,2022-01-11,-1


In [4]:
def to_submit(data,output):
    datetime=[]
    trade=[]
    signal=[]
    price=[]
    current=0
    for i in range(len(data.close)):
        if(i==(len(data.close)-1)):
            if(current==1):
                datetime.append(output.datetime[i])
#                 price.append(data.close[i])
                trade.append('close')
                signal.append(-1)
            elif(current==-1):
                datetime.append(output.datetime[i])
#                 price.append(data.close[i])
                trade.append('close')
                signal.append(1)
            else:
                break
            break
        if(output.signal[i]==1):
            if(current==1):
                datetime.append(output.datetime[i])
                trade.append('close')
#                 price.append(data.close[i])
                signal.append(0)
            elif(current==0):
                datetime.append(output.datetime[i])
#                 price.append(data.close[i])
                trade.append('close')
                signal.append(1)
                current=1
            else:
                datetime.append(output.datetime[i])
                trade.append('close')
#                 price.append(data.close[i])
                signal.append(1)
                current=0
            
        elif(output.signal[i]==-1):
            if(current==1):
                datetime.append(output.datetime[i])
                signal.append(-1)
#                 price.append(data.close[i])
                trade.append('close')
                current=0
            elif(current==0):
                datetime.append(output.datetime[i])
                trade.append('close')
#                 price.append(data.close[i])
                signal.append(-1)
                current=-1
            else:
                datetime.append(output.datetime[i])
                trade.append('close')
#                 price.append(data.close[i])
                signal.append(0)
                current=-1
        
        else:
            datetime.append(output.datetime[i])
            trade.append('close')
#             price.append(data.close[i])
            signal.append(0)
        
            
            
    to_submit=pd.DataFrame(columns=['datetime','signal','close\open'])
    to_submit['datetime']=datetime
    to_submit['signal']=signal
    to_submit['close\open']=trade
#     to_submit['price']=price
    return to_submit
            
            


def metrics(data,submit):
    capital=1000
    current=0
    temp=0
    portfolio_value=[]
    datetime=[]
    quantity=[]
    
    sharpe=[]
    returns=[]
    risk_free_rate=0.05       
    for i in range(len(data.close)):
        datetime.append(data.datetime[i])
        if(submit.signal[i]==1):
            if(current==0):
                portfolio_value.append(capital)
                returns.append((capital-1000)/10)
                sharpe.append((np.mean(returns)-risk_free_rate)/(np.std(returns)))
                
                temp=capital/(data.close[i])
                quantity.append(temp)
                capital=0
                current=1
            else:
                capital=capital-(temp*data.close[i])
                returns.append((capital-1000)/10)
                sharpe.append((np.mean(returns)-risk_free_rate)/(np.std(returns)))
                quantity.append(temp)
                portfolio_value.append(capital)
                temp=0
                current=0
                
        elif(submit.signal[i]==-1):
            if(current==0):
                portfolio_value.append(capital)
                temp=capital/(data.close[i])
                quantity.append(temp)
                returns.append((capital-1000)/10)
                sharpe.append((np.mean(returns)-risk_free_rate)/(np.std(returns)))
                capital=capital+(temp*data.close[i])
                
                current=-1
            else:
                capital=capital+(temp*data.close[i])
                quantity.append(temp)
                current=0
                temp=0
                portfolio_value.append(capital)
                returns.append((capital-1000)/10)
                sharpe.append((np.mean(returns)-risk_free_rate)/(np.std(returns)))
        
        else:
            if(current==0):
                portfolio_value.append(capital)
                quantity.append(temp)
                returns.append((capital-1000)/10)
                sharpe.append((np.mean(returns)-risk_free_rate)/(np.std(returns)))
            elif(current==1):
                portfolio_value.append(capital+(temp*data.close[i]))
                quantity.append(temp)
                returns.append((capital+(temp*data.close[i])-1000)/10)
                sharpe.append((np.mean(returns)-risk_free_rate)/(np.std(returns)))
            else:
                portfolio_value.append(capital-(temp*data.close[i]))
                quantity.append(temp)
                returns.append((capital-(temp*data.close[i])-1000)/10)
                sharpe.append((np.mean(returns)-risk_free_rate)/(np.std(returns)))
                
    a=pd.DataFrame(columns=['datetime','portfolio value'])
    a['datetime']=datetime
    a['portfolio value']=portfolio_value
#     a['sharpe']=sharpe
    a['quantity']=quantity
    
#     a['daily_return'] = a['portfolio value'].pct_change()
#     daily_volatility = a['daily_return'].std()
#     annual_volatility = daily_volatility * math.sqrt(252)
#     risk_free_rate = 0.03
    
#     a['excess_return'] = a['daily_return'] - risk_free_rate / 252
#     cumulative_excess_return = a['excess_return'].sum()
#     negative_returns = [r for r in a['excess_return'] if r < 0]
#     cumulative_volatility = a['daily_return'].std() * math.sqrt(252)
#     hello=np.std(negative_returns)*math.sqrt(252)
#     sharpe_ratio = cumulative_excess_return / cumulative_volatility
#     sortino_ratio=cumulative_excess_return / hello

    a=pd.DataFrame(columns=['datetime','portfolio value'])
    a['datetime']=datetime
    a['portfolio value']=portfolio_value
    a['quantity']=quantity
    a['daily_return'] = a['portfolio value'].pct_change()
    daily_volatility = a['daily_return'].std()
    annual_volatility = daily_volatility * math.sqrt(365)
    annual_volatility_negative = a[a['daily_return'] < 0]['daily_return'].std() * math.sqrt(365)
    risk_free_rate = 0.05
    a['excess_return'] = (a['daily_return'] - risk_free_rate)
    annual_excess_return = (a['daily_return'].mean()*365 - risk_free_rate)
    cumulative_excess_return = a['excess_return'].sum()
    negative_returns = [r for r in a['excess_return'] if r < 0]
    cumulative_volatility = a['daily_return'].std()*math.sqrt(252)
    hello=np.std(negative_returns)*math.sqrt(252)
    sharpe_ratio = annual_excess_return / annual_volatility
    sortino_ratio = annual_excess_return / annual_volatility_negative
    
    
    
    start=[]
    end=[]
    start_value=[]
    end_value=[]
    duration=[]
    P_and_L=[]
    return_for_every_trade=[]
    pointer=0
    drawdown=[]
    for i in range(len(submit.signal)):
        if(submit.signal[i]==1):
            if(pointer==0):
                start.append(i)
                start_value.append(portfolio_value[i])
                pointer=1
            else:
                end.append(i)
                end_value.append(portfolio_value[i])
                return_for_every_trade.append(100*((end_value[-1]/start_value[-1])-1))
                P_and_L.append(end_value[-1]-start_value[-1])
                duration.append(end[-1]-start[-1])
                pointer=0
                minimum=np.min(portfolio_value[start[-1]:end[-1]])
                maximum=np.max(portfolio_value[start[-1]:end[-1]])
                drawdown.append(100*((maximum-minimum)/maximum))
#                 drawdown.append(100*((portfolio_value[start[-1]]/minimum)-1))
                
        elif(submit.signal[i]==-1):
            if(pointer==0):
                start.append(i)
                start_value.append(portfolio_value[i])
                pointer=-1
            else:
                end.append(i)
                end_value.append(portfolio_value[i])
                return_for_every_trade.append(100*((end_value[-1]/start_value[-1])-1))
                P_and_L.append(end_value[-1]-start_value[-1])
                duration.append(end[-1]-start[-1])
                pointer=0
                minimum=np.min(portfolio_value[start[-1]:end[-1]])
                maximum=np.max(portfolio_value[start[-1]:end[-1]])
                drawdown.append(100*((maximum-minimum)/maximum))
                
    b=pd.DataFrame(columns=['start','end'])
    b['start']=start
    b['end']=end
    b['duration']=duration
    b['return for every trade']=return_for_every_trade
    b['P and L']=P_and_L
    b['drawdown']=drawdown
    
    

    gross_profit=[]
    gross_loss=[]
    for i in range(len(P_and_L)):
        if(P_and_L[i]>0):
            gross_profit.append(P_and_L[i])
        else:
            gross_loss.append(P_and_L[i])
    print("GROSS PROFIT: ",np.sum(gross_profit))
    print("GROSS LOSS: ",np.sum(gross_loss))
    print("NET P and L: ",np.sum(P_and_L))
    print("TOTAL CLOSED TRADES: ",len(start))
    print("WIN RATE: ",100*(len(gross_profit)/(len(gross_profit)+len(gross_loss))))
    print("AVG WINNING TRADE: ",np.mean(gross_profit))
    print("AVG LOSING TRADE: ",np.mean(gross_loss))
    print("Largest Losing trade: ",min(gross_loss))
    print("Largest winning trade: ",max(gross_profit))
    print("Avg holding duration: ",np.mean(duration))
    
   
    
    print("Sharpe Ratio: ",sharpe_ratio)


#     std_dev_negative = np.std(negative_returns)
            
    print("Sortino Ratio: ",sortino_ratio)
    print("RETURNS: ",((np.sum(P_and_L))/10))
    print("MAX Drawdown: (in %) ",np.max(drawdown))
    
    return a,b


def only_long_then_short(data,output):
    datetime=[]
    signal=[]
    trade=[]
    price=[]
    current=0
    for i in range(len(data.close)):
        datetime.append(data.datetime[i])
        trade.append('close')
#         price.append(data.close[i])
        if(i==len(data.close)-1):
            if(current==1):
                signal.append(-1)
                break
            else:
                signal.append(0)
                break
            
        
        if(output.signal[i]==1):
            if(current==0):
                signal.append(1)
                current=1
            else:
                signal.append(0)
        elif(output.signal[i]==-1):
            if(current==1):
                signal.append(-1)
                current=0
            else:
                signal.append(0)
            
        else:
            if(current==0):
                signal.append(0)
            else:
                signal.append(0)
    a=pd.DataFrame(columns=['datetime','signal','trade','price'])
    a['price']=price
    a['datetime']=datetime
    a['signal']=signal
    a['trade']=trade
    return a
            

In [5]:
c,d=metrics(data,to_submit(data,a))

C:\Users\tanus\AppData\Local\Temp\ipykernel_12540\1316258859.py:135: RuntimeWarning: divide by zero encountered in double_scalars
  sharpe.append((np.mean(returns)-risk_free_rate)/(np.std(returns)))
C:\Users\tanus\AppData\Local\Temp\ipykernel_12540\1316258859.py:96: RuntimeWarning: divide by zero encountered in double_scalars
  sharpe.append((np.mean(returns)-risk_free_rate)/(np.std(returns)))


GROSS PROFIT:  9945.94658806927
GROSS LOSS:  -2519.4681431989557
NET P and L:  7426.478444870313
TOTAL CLOSED TRADES:  26
WIN RATE:  50.0
AVG WINNING TRADE:  765.0728144668669
AVG LOSING TRADE:  -193.80524178453504
Largest Losing trade:  -1091.2478498687933
Largest winning trade:  3982.2219282943106
Avg holding duration:  53.15384615384615
Sharpe Ratio:  1.0694442959007024
Sortino Ratio:  1.4366515351679083
RETURNS:  742.6478444870313
MAX Drawdown: (in %)  81.93356249195011


In [6]:
c

,datetime,portfolio value,quantity,daily_return,excess_return
0,2018-01-01,1000.000000,0.000000,NaN,NaN
1,2018-01-02,1000.000000,0.000000,0.000000,-0.050000
2,2018-01-03,1000.000000,0.000000,0.000000,-0.050000
3,2018-01-04,1000.000000,0.000000,0.000000,-0.050000
4,2018-01-05,1000.000000,0.000000,0.000000,-0.050000
...,...,...,...,...,...
1468,2022-01-08,8659.442772,0.116764,0.014054,-0.035946
1469,2022-01-09,8509.148216,0.116764,-0.017356,-0.067356
1470,2022-01-10,8551.464499,0.116764,0.004973,-0.045027
1471,2022-01-11,8417.137360,0.116764,-0.015708,-0.065708


In [7]:
d

,start,end,duration,return for every trade,P and L,drawdown
0,60,72,12,-24.379674,-243.796740,23.651330
1,73,113,40,-15.813315,-119.580806,24.420987
2,116,141,25,-11.011939,-70.104478,17.542495
3,142,197,55,1.791027,10.146489,19.989642
4,198,219,21,-15.681957,-90.432274,16.269249
5,223,245,22,-15.568103,-75.697129,19.323928
6,246,255,9,-11.903458,-48.867868,15.924643
7,256,413,157,39.785426,143.890834,37.140866
8,414,567,153,160.579187,811.820970,73.244907
9,568,582,14,-16.216982,-213.639112,22.316075


In [8]:
e,f=metrics(data,only_long_then_short(data,a))

C:\Users\tanus\AppData\Local\Temp\ipykernel_12540\1316258859.py:135: RuntimeWarning: divide by zero encountered in double_scalars
  sharpe.append((np.mean(returns)-risk_free_rate)/(np.std(returns)))
C:\Users\tanus\AppData\Local\Temp\ipykernel_12540\1316258859.py:96: RuntimeWarning: divide by zero encountered in double_scalars
  sharpe.append((np.mean(returns)-risk_free_rate)/(np.std(returns)))


GROSS PROFIT:  7930.259038264578
GROSS LOSS:  -882.0352772210439
NET P and L:  7048.223761043534
TOTAL CLOSED TRADES:  13
WIN RATE:  53.84615384615385
AVG WINNING TRADE:  1132.894148323511
AVG LOSING TRADE:  -147.00587953684064
Largest Losing trade:  -243.79674041864382
Largest winning trade:  4954.530673118177
Avg holding duration:  56.30769230769231
Sharpe Ratio:  1.16501001621746
Sortino Ratio:  1.114139535903099
RETURNS:  704.8223761043535
MAX Drawdown: (in %)  82.39065231779212


In [9]:
e

,datetime,portfolio value,quantity,daily_return,excess_return
0,2018-01-01,1000.000000,0.0,NaN,NaN
1,2018-01-02,1000.000000,0.0,0.0,-0.05
2,2018-01-03,1000.000000,0.0,0.0,-0.05
3,2018-01-04,1000.000000,0.0,0.0,-0.05
4,2018-01-05,1000.000000,0.0,0.0,-0.05
...,...,...,...,...,...
1468,2022-01-08,8048.223761,0.0,0.0,-0.05
1469,2022-01-09,8048.223761,0.0,0.0,-0.05
1470,2022-01-10,8048.223761,0.0,0.0,-0.05
1471,2022-01-11,8048.223761,0.0,0.0,-0.05


In [10]:
f

,start,end,duration,return for every trade,P and L,drawdown
0,60,72,12,-24.379674,-243.796740,23.651330
1,113,141,28,-12.714922,-96.150652,17.542495
2,197,219,22,-14.706875,-97.073115,16.269249
3,245,255,10,-11.012376,-61.997418,15.924643
4,413,567,154,163.875950,820.989133,73.244907
5,582,597,15,-14.051293,-185.754052,15.283532
6,669,689,20,-17.361408,-197.263300,13.223432
7,736,788,52,6.245824,58.645409,24.585083
8,828,907,79,24.888995,248.292427,31.759825
9,935,976,41,11.170134,139.167777,22.155763
